In [1]:
# Import libraries
import pandas as pd
import urllib
import json

In [2]:
# Calculates centroid of a convex polygon from a list of vertices
def centroid(vertices):
     _x_list = [vertex [0] for vertex in vertices]
     _y_list = [vertex [1] for vertex in vertices]
     _len = len(vertices)
     _x = sum(_x_list) / _len
     _y = sum(_y_list) / _len
     return(_x, _y)

In [3]:
# Read in Airports dataset and keep necessary columns
airports_df = pd.read_csv('Airports.csv')
airports_df = airports_df[['GLOBAL_ID','IDENT','ICAO_ID']]

In [4]:
# Pass request to site for GeoJSON
runway_url = "https://opendata.arcgis.com/datasets/4d8fa46181aa470d809776c57a8ab1f6_0.geojson"
req = urllib.request.urlopen(runway_url)

In [5]:
# Read into JSON
data = req.read()
runways_json = json.loads(data.decode('utf-8'))

In [6]:
# Read JSON into dataframe
runways_df = pd.json_normalize(runways_json, record_path =['features'])
# Add lat/long for runway center
runways_df['RUNWAY.LATITUDE'] = ''
runways_df['RUNWAY.LONGITUDE'] = ''

In [7]:
# Extract list of coordinates from nested list
for i in runways_df.index:
    runways_df.at[i, 'geometry.coordinates'] = runways_df['geometry.coordinates'][i][0]

In [8]:
# Iterate through dataframe to calculate center of runway and split into lat/long values
for i in runways_df.index:
    coords = centroid(runways_df['geometry.coordinates'][i])
    runways_df.at[i, 'RUNWAY.LATITUDE'] = coords[0]
    runways_df.at[i, 'RUNWAY.LONGITUDE'] = coords[1]

In [9]:
# Remove unnecessary rows and rename for readability
runways_df = runways_df[['properties.GLOBAL_ID','properties.AIRPORT_ID','properties.DESIGNATOR','RUNWAY.LATITUDE','RUNWAY.LONGITUDE']]
runways_df = runways_df.rename({'properties.GLOBAL_ID': 'GLOBAL_ID','properties.AIRPORT_ID': 'AIRPORT_ID','properties.DESIGNATOR': 'DESIGNATOR'}, axis=1)

In [10]:
# Inner join of Runways and Airports on Runways.AIRPORT_ID and Airports.GLOBAL_ID
runways_df = pd.merge(runways_df, airports_df, how='inner', left_on = 'AIRPORT_ID', right_on = 'GLOBAL_ID')

In [11]:
# Remove unnecessary rows for readability
runways_df = runways_df[['DESIGNATOR','RUNWAY.LATITUDE','RUNWAY.LONGITUDE','IDENT','ICAO_ID']]

In [12]:
runways_df

,DESIGNATOR,RUNWAY.LATITUDE,RUNWAY.LONGITUDE,IDENT,ICAO_ID
0,05/23,-176.645,51.8825,ADK,PADK
1,04/22,-154.182,56.939,AKK,PAKH
2,01/19,-161.493,60.9146,Z13,PFZK
3,E/W,-0.19998,-8.06404e-05,KKI,NaN
4,NW/SE,-0.19998,-8.06404e-05,KKI,NaN
...,...,...,...,...,...
24031,176/356,-99.2656,34.6767,LTS,KLTS
24032,18L/36R,-99.2596,34.6677,LTS,KLTS
24033,18R/36L,-99.2739,34.6603,LTS,KLTS
24034,01/19,-84.9466,36.3507,2A1,NaN


In [13]:
# Export dataframe to CSV
runways_df.to_csv('runways_cleaned.csv')